In [1]:
import pandas as pd
import numpy as np
import requests
import os
from urllib.request import urlopen
from urllib import request
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from selenium import webdriver
import socket
import time
import ssl
df = pd.read_csv("excel_urls.csv",header=None,delimiter="\t")

def get_driver_url_content(url, encoding='utf-8', timeout=3):
    '''
    使用浏览器获取动态内容
    :param url:         网页url
    :param encoding:    网页编码
    :param timeout:     设置超时
    :return:
    '''
    
    driver = webdriver.Chrome()
    # 也可以使用phantomJS
    # driver =webdriver.Phantomjs(executable_path="/path/to/phantomjs")
    driver.get(url)
    time.sleep(timeout)
    try:
        files = driver.find_elements_by_xpath("//*[contains(text(), 'docx')]")
        for i in files:
            if i.text!="":
                file = i
    
        href = file.get_attribute("href")
        driver.close()
        
    except:
        driver.close()
    return href    



def download(url,name,savepath='./'):
    """
    download file from internet
    :param url: path to download from
    :param savepath: path to save files
    :return: None
    """
    def reporthook(a, b, c):
        """
        显示下载进度
        :param a: 已经下载的数据块
        :param b: 数据块的大小
        :param c: 远程文件大小
        :return: None
        """
        print("\rdownloading: %5.1f%%" % (a * b * 100.0 / c), end="")
    # 判断文件是否存在，如果不存在则下载
    filename = name+".docx"
    if not os.path.isfile(os.path.join(savepath, filename)):
        print('Downloading data from %s' % url)
        urlretrieve(url, os.path.join(savepath, filename), reporthook=reporthook)
        print('\nDownload finished!')
    else:
        print('File already exsits!')
    # 获取文件大小
    filesize = os.path.getsize(os.path.join(savepath, filename))
    # 文件大小默认以Bytes计， 转换为Mb
    print('File size = %.2f Mb' % (filesize/1024/1024))
if __name__ == '__main__':
    # 以下载cifar-10数据集为例
    ssl._create_default_https_context = ssl._create_unverified_context
    urls = df[4].values
    names = df[0].values
    for i in range(len(urls)):
        url = urls[i]
        filename = str(i)
        try:
            a = get_driver_url_content(url)
            download(a,filename,savepath='./docx/')
        except:
            print("Skipping this, need special attention for "+url)
            print(names[i])

downloading: 111.4%
Download finished!
File size = 0.04 Mb
downloading: 100.9%
Download finished!
File size = 0.04 Mb
downloading: 107.4%
Download finished!
File size = 0.04 Mb
downloading: 752.9%
Download finished!
File size = 0.00 Mb
Skipping this, need special attention for http://news.szhome.com/338103.html
首批房产中介黑名单
Skipping this, need special attention for https://3g.163.com/house/article/FHL73QCA00078224.html
第二批批房产中介黑名单
Skipping this, need special attention for https://m.credit100.com/xhxy/c/2020-07-21/624567.shtml?title=%E6%96%B0%E5%8D%8E%E4%BF%A1%E7%94%A8-%E4%BF%A1%E7%94%A8%E5%A4%B4%E6%9D%A1&from=groupmessage&isappinstalled=0
首次发布环评信用平台失信黑名单
Skipping this, need special attention for https://mp.weixin.qq.com/s/ODgW4cUU6QZvngW8VpIktQ
全国人力资源诚信服务示范机构候选名单
Skipping this, need special attention for https://mp.weixin.qq.com/s/qNsQmC4fN9S1mbsAA3OdiQ
银行涉企违规收费名单
Skipping this, need special attention for https://mp.weixin.qq.com/s/VMr3X1g_psMkcze_6Vlj6Q
重大违法违规股东名单
Skipping this, need spe

In [267]:
from ftplib import FTP 
import cx_Oracle as oracle
import pandas as pd
import re
#print(len(doc.tables[0].rows))
def doc_to_df(doc):
    pattern2 = re.compile(r"博客")
    pattern = re.compile(r"类别|表现形式|[1-9]+|序号")
    row_nums = []
    row_type = []
    tb = doc.tables[0]
    for row_num in range(len(tb.rows)):
        row_title = tb.rows[row_num].cells[0].text
        if bool(pattern.search(row_title))==False:
            row_nums.append(row_num+1)
            row_type.append(row_title)
    
    row_nums.append(len(tb.rows))
    for i in range(len(row_nums)-1):
        find_df(tb,row_nums[i],row_nums[i+1]-1)

def data_output(data,dataTable):
    connection = db
    cursor = connection.cursor()
    query = "INSERT INTO "+ dataTable + " VALUES ({})"
    columns = list(data.columns)
    aidx = list(range(1,len(columns)+1))
    aidx = [':'+str(i) for i in aidx]
    aname = ','.join(aidx)
    dtHigh = data.shape[0]
    dtWidth = data.shape[1]
    creatVar = locals()
    wholeData = []
    for i in range(dtHigh):
        value_list = []
        for j in range(dtWidth):
            value_list.append("{}".format(str(data.iloc[i,j])))
        wholeData.append(value_list)
    cursor.executemany(query.format(aname),wholeData)
    connection.commit()
    cursor.close()
    return
import numpy as np
def find_df(tb,row1,row2):
    index,bad_urls,bad_type,date,descriptions=[],[],[],[],[]
    table_index=0
    pattern = re.compile(r"(http.+|www.+)")
    pattern2 = re.compile(r"(.+)http:/.+")
    pattern3 = re.compile(r"\“(.+)”")
    pattern4 = re.compile(r"\：(.+)http")
    
    for row in tb.rows[row1:row2]:
        text= ""
        for p in row.cells[2].paragraphs:##如果cell中有多段，即有回车符
            text+=p.text
        try:
            url = re.findall(pattern,text)[0]
        except:
            url = None

        if url == None:
            continue
        bad_type.append(row.cells[3].text)         
        bad_urls.append(url)
        descriptions.append(row.cells[1].text)
        date.append(row.cells[4].text)
        index.append(row.cells[0].text)#这行也可以放在else中

    df=pd.DataFrame({"index":index,'date':date,"bad_url":bad_urls,"bad_type":bad_type,"description":descriptions})
    df = df.replace(dict_type)
    data_output(df,"bad_website")

def get_stock_df(doc):
    
    tb = doc.tables[0]
    index,stock_num,iname,start,end = [],[],[],[],[]
    for row in tb.rows[1:]:
        
        cells = row.cells
        index.append(cells[0].text)
        stock_num.append(cells[1].text)
        iname.append(cells[2].text)
        start.append(cells[3].text.replace("年","-").replace("月","-").replace("日","-"))
        end.append(cells[4].text.replace("年","-").replace("月","-").replace("日","-"))
    df = pd.DataFrame({"row_index":index,"stock_name":stock_num,"iname":iname,"start_date":start,"end_date":end})
    data_output(df,"stock_peishou")
    return df
    

if __name__ == '__main__':
    username = input("Username(必须大写）:   ")
    password = input("Password:   ")
    database_ip = input("数据库IP(默认127.0.0.1):   ")
    port = input("Port(默认1521):    ")
    database = input("服务名/数据库(默认orcl):    ")
    string = username+"/"+password+"@"+database_ip+":"+"port"+"/"+database
    try:
        db = oracle.connet(string)
    except:
        db = oracle.connect('tenant01/123456@127.0.0.1:1521/orcl')
    dict_df = pd.read_csv("type_dict.csv")
    dict_type = dict(zip(dict_df["type"],dict_df["description"]))
    a = [0,1,2]+list(range(14,26))

    for i in a:
        file = str(i)+".docx"
        doc_path = os.path.join(os.getcwd(), "docx\\"+file)
        print(doc_path)
        document = docx.Document(doc_path)
        doc_to_df(document)
    doc_path = os.path.join(os.getcwd(), "docx\\"+"13.docx")
    doc = docx.Document(doc_path)
    get_stock_df(doc)
    db.close()
        


In [2]:
# 其中 0，1，2，14-25 是同一类数据，13是一类数据（含有表格的docx文件），其他的都需要特殊处理